In [11]:
import os
# os.environ['SCOPE'] = 'uat'
# os.environ['CUSTOMER'] = 'CSIO'
# os.environ['NameSpace'] = 'airflow'
if os.environ['CUSTOMER'] == 'CSIO':
    %run /home/jovyan/work/DAGS_Airflow/icm_config_csio.ipynb
elif os.environ['CUSTOMER'] == 'globelife':
    %run /home/jovyan/work/Tanish/icm_config.ipynb
elif os.environ['CUSTOMER'] == 'regeneron':
    %run /home/jovyan/work/DAGS_Airflow/icm_config_regeneron.ipynb

In [12]:
%run /home/jovyan/work/DAGS_Airflow/sparkConf.ipynb

In [13]:
Activity = os.environ['CUSTOMER']
Activity

'CSIO'

In [14]:
import os
# Set executor python spark environments
os.environ['PYSPARK_PYTHON'] = "/miniconda/bin/python"
from utils.spark_util import KubeflowSparkUtil
sparkConf = KubeflowSparkUtil.get_spark_conf(KubeflowSparkUtil, spark_conf, 'AiopsModeling-Airflow-'+Activity, 'k8s://https://kubernetes.default.svc.cluster.local:443')
from pyspark.sql import SQLContext
spark=  KubeflowSparkUtil(sparkConf).get_spark_session()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

In [4]:
%run /home/jovyan/work/DAGS_Airflow/aiops_util.ipynb

In [5]:
from utils.customer import FetchCustomer
from utils.aiops_config import AiopsConfig
from core.icm.modeling import ICMDeviceSelector, ProphetModelGenerator
from core.icm.compute_recommendation_generation import ICMComputeRecommendation
from core.icm.storage_recommendation_generation import ICMStorageRecommendation
from core.icm.baseline_summary import BaselineSummary
from core.icm.icm_ranking import execute_rank
from core.icm.recommendation_helper import ICMRecommendationHelper
from core.icm.device_details import ComputeDevices, StorageDevices

In [6]:
customer = os.environ['CUSTOMER']
aiopsconfig = AiopsConfig(customer,2, icm_configurations,icm_modelDBconfig)

In [7]:
icm_configurations

{'globelife': {'device_selection_days': 100,
  'predict_days': 30,
  'last_month_days_count': 14,
  'db_server_name': 'ucareserverprod.database.windows.net',
  'db_port': 1433,
  'database_name': 'ucare_dw_prod',
  'service_check_format': False,
  'multiple_tables': True,
  'MDL_avaliable': False,
  'CMDB_avaliable': True,
  'allocation_available': False,
  'allocation_as_usage': True,
  'allocation_in bytes': False,
  'allocation_in mb': True,
  'allocation_in kb': False,
  'icm_recommendation_automation': False,
  'icm_recommendation_automation_choice': {'cpu_mem_rank': ['Downscale 1',
    'Upscale 1'],
   'disk_rank': ['Upscale 1']},
  'icm_recommendation_automation_limit': 10,
  'icm_kafka_topic': 'aiops.recommendation',
  'icm_kafka_servers': '<ICM_KAFKA_SERVER_NAME>',
  'cluster_flag': False,
  'sr_query': "select approval,number,dv_state,dv_cat_item,dv_context,opened_at from ods_sn_uni..sc_req_item (nolock) where dv_cat_item = 'AIOps'",
  'request_item_query': 'select m.sys_upda

In [8]:
device_selector = ICMDeviceSelector(aiopsconfig,spark, True)
compute_utilization_df, compute_devices, storage_utilization_df, storage_devices, ci_exclusion_compute_list, ci_exclusion_disk_list, iops_throughput_df = device_selector.select_icm_device()

key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant-id
secret:  8d894c2b-238f-490b-8dd1-d93898c5bf83
key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant-id
secret:  8d894c2b-238f-490b-8dd1-d93898c5bf83
key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant-id
secret:  8d894c2b-238f-490b-8dd1-d93898c5bf83
key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant-id
secret:  8d894c2b-238f-490b-8dd1-d93898c5bf83
key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant

KeyboardInterrupt: 

In [ ]:
print(compute_devices)

In [61]:
model_generator = ProphetModelGenerator(aiopsconfig,spark)
cpu_finaldf = model_generator.generate_forecast('CPU',compute_utilization_df,compute_devices,True)
mem_finaldf = model_generator.generate_forecast('MEM',compute_utilization_df,compute_devices,True)
disk_finaldf = model_generator.generate_forecast('DISK',storage_utilization_df,storage_devices,True)

key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant-id
secret:  8d894c2b-238f-490b-8dd1-d93898c5bf83
key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant-id
secret:  8d894c2b-238f-490b-8dd1-d93898c5bf83
key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant-id
secret:  8d894c2b-238f-490b-8dd1-d93898c5bf83


/usr/local/spark/python/pyspark/sql/pandas/group_ops.py:81: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant-id
secret:  8d894c2b-238f-490b-8dd1-d93898c5bf83
key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant-id
secret:  8d894c2b-238f-490b-8dd1-d93898c5bf83
key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant-id
secret:  8d894c2b-238f-490b-8dd1-d93898c5bf83
key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant-id
secret:  8d894c2b-238f-490b-8dd1-d93898c5bf83
key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant

In [62]:
# MAGIC %md ###Compute Recommendation Generation

# COMMAND ----------
from core.icm.compute_recommendation_generation import ICMComputeRecommendation
compute_recommendation = ICMComputeRecommendation(cpu_finaldf, mem_finaldf, compute_utilization_df, aiopsconfig, spark, True)
compute_recommendation_df = compute_recommendation.generate_recommendation()

key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant-id
secret:  8d894c2b-238f-490b-8dd1-d93898c5bf83
key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant-id
secret:  8d894c2b-238f-490b-8dd1-d93898c5bf83
key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant-id
secret:  8d894c2b-238f-490b-8dd1-d93898c5bf83
key:  service-principal-id
secret:  b5c2e137-1f67-426c-871b-b67024e0d229
key:  service-principal-secret
secret:  ~d5MS_fL-N62CK666dQ~~QE1Ui323Mej-s
key:  tenant-id
secret:  8d894c2b-238f-490b-8dd1-d93898c5bf83


In [ ]:
# MAGIC %md ###Storage Recommendation Generation

# COMMAND ----------
from core.icm.storage_recommendation_generation import ICMStorageRecommendation
storage_recommendation = ICMStorageRecommendation(disk_finaldf, storage_utilization_df, aiopsconfig, spark,iops_throughput_df)
storage_recommendation_df = storage_recommendation.generate_recommendation()

In [ ]:
# MAGIC %md ###Baseline Summary

# COMMAND ----------
from core.icm.baseline_summary import BaselineSummary
baseline_summary = BaselineSummary(aiopsconfig,spark)
baseline_summary.summarize_baseline(compute_recommendation_df, spark.createDataFrame(storage_recommendation_df))

In [ ]:
# MAGIC %md ###Ranking

# COMMAND ----------
from core.icm.icm_ranking import execute_rank
compute_recommendation_df, compute_rank_range_df = execute_rank(aiopsconfig, spark, compute_recommendation_df.toPandas(), is_compute=True, primary_recom=True)


In [29]:
storage_recommendation_df, storage_rank_range_df = execute_rank(aiopsconfig, spark, storage_recommendation_df, is_compute=False)

In [71]:
# MAGIC %md ###Master recommendation table

# COMMAND ----------

#primary_recommendation = true by default, for secondary change to False 

recommendation_helper = ICMRecommendationHelper(aiopsconfig,spark)
recommendation_helper.form_master_recommendations(compute_recommendation_df,storage_recommendation_df, True)


In [72]:
# MAGIC %md ###Sufficient & In-sufficient Device details

# COMMAND ----------

#primary_recommendation = true by default, for secondary change to False 
compute_device_details = ComputeDevices(aiopsconfig, spark, True)
compute_device_details.get_sufficient_devices(compute_utilization_df,compute_devices)
compute_device_details.get_insufficient_devices(compute_utilization_df,compute_devices,ci_exclusion_compute_list)

In [73]:
# COMMAND ----------

#storage will call once

storage_device_details = StorageDevices(aiopsconfig,spark)
storage_device_details.get_sufficient_devices(storage_utilization_df,storage_devices)
storage_device_details.get_insufficient_devices(storage_utilization_df,storage_devices,ci_exclusion_disk_list)

In [74]:
# MAGIC %md ###Device Selection (for secondary metric)

# COMMAND ----------

#primary_recommendation = true by default, for secondary change to False 

device_selector = ICMDeviceSelector(aiopsconfig,spark, False)
compute_utilization_df, compute_devices, storage_utilization_df, storage_devices, ci_exclusion_compute_list, ci_exclusion_disk_list, iops_throughput_df = device_selector.select_icm_device()

In [ ]:
# MAGIC %md ###Model Generation (for secondary metric)

# COMMAND ----------

#primary_recommendation = true by default, for secondary change to False 

model_generator = ProphetModelGenerator(aiopsconfig,spark)
cpu_finaldf = model_generator.generate_forecast('CPU',compute_utilization_df,compute_devices,False)
mem_finaldf = model_generator.generate_forecast('MEM',compute_utilization_df,compute_devices,False)

In [ ]:
# MAGIC %md ###Compute Recommendation Generation (for secondary metric)

# COMMAND ----------

#primary_recommendation = true by default, for secondary change to False 

compute_recommendation = ICMComputeRecommendation(cpu_finaldf, mem_finaldf, compute_utilization_df, aiopsconfig, spark, False)
compute_recommendation_df = compute_recommendation.generate_recommendation()

In [ ]:
# MAGIC %md ###Ranking (for secondary metric)

# COMMAND ----------

#primary_recommendation = true by default, for secondary change to False 

compute_recommendation_df, compute_rank_range_df = execute_rank(aiopsconfig, spark, compute_recommendation_df.toPandas(), is_compute=True, primary_recom=False)


In [ ]:
# MAGIC %md ###Master recommendation table (for secondary metric)

# COMMAND ----------

#primary_recommendation = true by default, for secondary change to False 

recommendation_helper = ICMRecommendationHelper(aiopsconfig,spark)
recommendation_helper.form_master_recommendations(compute_recommendation_df,storage_recommendation_df, False)

In [ ]:
# MAGIC %md ###Master recommendation table (for secondary metric)

# COMMAND ----------

#primary_recommendation = true by default, for secondary change to False 

compute_device_details = ComputeDevices(aiopsconfig, spark, False)
compute_device_details.get_sufficient_devices(compute_utilization_df,compute_devices)
compute_device_details.get_insufficient_devices(compute_utilization_df,compute_devices,ci_exclusion_compute_list)

In [15]:
spark.stop()
#spark

In [10]:
spark

AttributeError: 'NoneType' object has no attribute 'sc'